In [2]:
import numpy as np
import cv2
from pathlib import Path
import PIL.Image as Image
import os,glob
from tqdm.notebook import tqdm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import matplotlib.pylab as plt
import tensorflow as tf 
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential


In [3]:
IMAGE_SHAPE = (224, 224)

import os
os.environ["TFHUB_CACHE_DIR"] = "./tfhub_cache"


classifier = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4", input_shape=IMAGE_SHAPE+(3,))
])


In [4]:
IMAGE_SHAPE+(3,)

(224, 224, 3)

In [5]:
base_dir = Path('plant-village')

X = []
y = []

# Iterate through each subdirectory (representing a class)
for class_dir in tqdm(base_dir.iterdir(), desc="Classes"):
    if class_dir.is_dir():
        label = class_dir.name  # e.g. "Potato___Early_blight"
        print(f"Loading class: {label}")

        # Iterate through images in this class folder
        for img_path in tqdm(list(class_dir.glob("*.jpg")), desc=f"Images in {label}", leave=False):
            img = cv2.imread(str(img_path))
            if img is not None:
                img = cv2.resize(img, (224, 224))
                X.append(img)
                y.append(label)  # Use label instead of slicing filename


Classes: 0it [00:00, ?it/s]

Loading class: Potato___Early_blight


Images in Potato___Early_blight:   0%|          | 0/1000 [00:00<?, ?it/s]

Loading class: Potato___healthy


Images in Potato___healthy:   0%|          | 0/152 [00:00<?, ?it/s]

Loading class: Potato___Late_blight


Images in Potato___Late_blight:   0%|          | 0/1000 [00:00<?, ?it/s]

In [6]:
CLASS_NAMES = ["Early blight", "Healthy", "Late blight"]
CLASS_NAMES

['Early blight', 'Healthy', 'Late blight']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)
print ("Shape of an image in X_train: ", X_train[0].shape)
print ("Shape of an image in X_test: ", X_test[0].shape)


Shape of an image in X_train:  (224, 224, 3)
Shape of an image in X_test:  (224, 224, 3)


In [8]:
le = preprocessing.LabelEncoder()
y = le.fit_transform(y)  # <- all at once

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

# Convert to numpy arrays
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)  # Shape: (N,)
y_test = np.array(y_test)



In [9]:
print("X_train Shape: ", X_train.shape)
print("X_test Shape: ", X_test.shape)
print("y_train Shape: ", y_train.shape)
print("y_test Shape: ", y_test.shape)

X_train Shape:  (1441, 224, 224, 3)
X_test Shape:  (711, 224, 224, 3)
y_train Shape:  (1441,)
y_test Shape:  (711,)


In [10]:
X_train_scaled = X_train / 255.0
X_test_scaled = X_test / 255.0


In [11]:

feature_extractor_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
pretrained_model_without_top_layer = hub.KerasLayer(
    feature_extractor_model, input_shape=(224, 224, 3), trainable=False)

In [16]:
num_of_classes = 3

model = tf.keras.Sequential([
    pretrained_model_without_top_layer,
    tf.keras.layers.Dense(num_of_classes, activation='softmax')
])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 1280)              2257984   
                                                                 
 dense_1 (Dense)             (None, 3)                 3843      
                                                                 
Total params: 2261827 (8.63 MB)
Trainable params: 3843 (15.01 KB)
Non-trainable params: 2257984 (8.61 MB)
_________________________________________________________________


In [17]:

model.compile(
    optimizer="adam",
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=["acc"])

model.fit(X_train_scaled, y_train, epochs=5)

Epoch 1/5
46/46 [==============================] - 86s 1s/step - loss: 0.5429 - acc: 0.8022
Epoch 2/5
46/46 [==============================] - 51s 1s/step - loss: 0.2140 - acc: 0.9410
Epoch 3/5
46/46 [==============================] - 49s 1s/step - loss: 0.1581 - acc: 0.9570
Epoch 4/5
46/46 [==============================] - 50s 1s/step - loss: 0.1263 - acc: 0.9639
Epoch 5/5
46/46 [==============================] - 49s 1s/step - loss: 0.1110 - acc: 0.9688


INFO:tensorflow:Assets written to: ../saved_models/crops\assets


INFO:tensorflow:Assets written to: ../saved_models/crops\assets


In [15]:
model.save('../saved_models/cr')

INFO:tensorflow:Assets written to: ../saved_models/cr\assets


INFO:tensorflow:Assets written to: ../saved_models/cr\assets


In [18]:
model.save('../saved_models/crop')

INFO:tensorflow:Assets written to: ../saved_models/crop\assets


INFO:tensorflow:Assets written to: ../saved_models/crop\assets


In [19]:
model.save('bforetf')

INFO:tensorflow:Assets written to: bforetf\assets


INFO:tensorflow:Assets written to: bforetf\assets


In [20]:
converter = tf.lite.TFLiteConverter.from_saved_model("bforetf")
tflite_model = converter.convert()

with open("model.tflite", "wb") as f:
    f.write(tflite_model)